## Sparse Recovery via Least Squares-Constrained Generalized Norm Minimization (LSCGNM)

This notebook applies **LSCGNM** to extract a sparse time-frequency representation of a short segment
from the classical piece *Für Elise*. It reconstructs sparse coefficients for a piano-frequency dictionary
under a least squares constraint using proximal optimization and binary search.


In [1]:
import numpy as np
from scipy.io import wavfile

### Audio Preprocessing
- Load and normalize the audio file (*Für Elise*)
- Segment the waveform into short 30ms blocks using `get_audio_chunks`


In [2]:
def rescale_audio(data):
    if data.min() < -1 or data.max() > 1:
        data = data / max(abs(data.min()), abs(data.max()))

    return data

def get_audio_chunks(data, chunk_size):
    for i in range(0, len(data), chunk_size):
        yield data[i:i + chunk_size]


### Optimization Functions
- `grad_least_squares`: Computes gradient of the least squares objective \( \nabla_x \|Ax - y\|^2 \)
- `prox12norm`: Proximal operator for group-sparse 1-2 norm
- `apgd`: Accelerated Proximal Gradient Descent algorithm


In [4]:
def grad_least_squares(A, x, y):
    return A.T @ (A @ x - y)

def prox12norm(x, gamma, d=2):
    x = x.reshape(-1, d)
    norm = np.linalg.norm(x, axis=1, keepdims=True)
    factor = np.maximum(1 - gamma / norm, 0)
    x = factor * x

    return x.reshape(-1)


def apgd(gradf, proxf, gamma, L, x, tol=1e-6, maxit=1e5):
    # x = x0
    z = x.copy()
    t = 1
    step_size = 1 / L
    for i in range(maxit):
        # grad step
        grad = gradf(z)
        x_next = proxf(z-step_size*grad, gamma*step_size)
        # momentum step
        t_new = (1 + np.sqrt(1 + 4 * t ** 2)) / 2
        z_next = x_next + ((t - 1) / t_new) * (x_next - x)

        if np.linalg.norm(x_next-x) <= tol:
            # print(z)
            return x_next,i
        t = t_new
        x = x_next
        z = z_next


        
    # print(z)
    return x, maxit+1

### Frequency Dictionary
- `get_piano_keys`: Generates the fundamental frequencies for the 88 piano keys
- `get_A`: Constructs a matrix of sine and cosine basis vectors using piano frequencies


In [5]:
def get_piano_keys():
    keys = [27.5 * (2 ** (i / 12)) for i in range(88)]
    return keys

def get_A(frequencies, f_s, l):
    A = np.zeros((l, 2*len(frequencies)))
    for i, f in enumerate(frequencies):
        for j in range(l):
            A[j, 2 * i] = np.cos(2 * np.pi * f * j / f_s)
            A[j, 2 * i + 1] = np.sin(2 * np.pi * f * j / f_s)
    return A



### LSCGNM Solver with Bisection
- `lscgnm`: Solves a constrained recovery problem:
\[ \min_x \|x\|_{1,2} \quad \text{subject to} \quad \|Ax - y\|_2 \leq \sigma \]
- Uses binary search to tune the regularization parameter `γ` such that the constraint is satisfied
- Leverages APGD as the inner solver for each γ trial


In [6]:
def lscgnm(A, y, sigma, d, tol, gnrls_x0, gnrls_tol, gnrls_maxit):
    L = np.linalg.norm(A.T @ A, ord=2)
    gamma0 = 1e-6
    x, _ = apgd(lambda x: grad_least_squares(A, x, y), lambda x, t: prox12norm(x, t, d), gamma0, L, gnrls_x0, gnrls_tol, gnrls_maxit)
    sigma0 = np.linalg.norm(A @ x - y)
    # print(sigma0)
    sigma_max = np.linalg.norm(y)
    # print(sigma_max)
    if sigma >= sigma0:
        if sigma >= sigma_max:
            x = np.zeros(A.shape[1])
            gamma_opt = np.Infinity
            return x, gamma_opt
        else:
            gamma_u = 1
            while True:
                x, _ = apgd(lambda x: grad_least_squares(A, x, y), lambda x, t: prox12norm(x, t, d), gamma_u, L, gnrls_x0, gnrls_tol, gnrls_maxit)
                norm = np.linalg.norm(A @ x - y)

                if norm > sigma:
                    gamma_opt = gamma_u
                    break
                gamma_u = 2 * gamma_u
            
            #bisection
            gamma_l = 0
            num_iterations = 0
            while gamma_u - gamma_l > tol:
                gamma_mid = (gamma_l + gamma_u) / 2
                x_opt, _ = apgd(lambda x: grad_least_squares(A, x, y), lambda x, t: prox12norm(x, t, d), gamma_mid, L, gnrls_x0, gnrls_tol, gnrls_maxit)
                error = np.linalg.norm(A @ x_opt - y)

                if error < sigma:
                    gamma_l = gamma_mid
                else:
                    gamma_u = gamma_mid

                num_iterations += 1

            gamma_star = (gamma_l + gamma_u) / 2
            return x_opt, gamma_star
                
    else:
        gamma_opt = -1
        return None, gamma_opt


### Driver Function for Recovery Across σ Values
- `lscgnm_driver`: Runs the LSCGNM recovery for a single audio block
- Sweeps over a list of σ values to study the trade-off between error tolerance and sparsity


In [7]:
def lscgnm_driver(block, f_s, l,d=2, tol=1e-3, gnrls_tol = 1e-6, gnrls_maxit=10**5):

    f = get_piano_keys()
    A = get_A(f, f_s, l)
    L = np.linalg.norm(A.T @ A, ord=2)
    result = []
    x0 = np.zeros(A.shape[1])
    sigma_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    d = 2
    for i, sigma in enumerate(sigma_list):
        x, gamma = lscgnm(A, block, sigma, d, tol, x0, gnrls_tol, gnrls_maxit)
        result.append(x)
        print(f"| sigma: {sigma} | gamma: {gamma} |")
    return result


### Example Execution
- Load audio and segment it
- Select one block (e.g., 349th)
- Run LSCGNM driver to observe how γ changes with σ


In [8]:
samplerate, data_orig = wavfile.read('furelise.wav')
data = data_orig / np.max(np.abs(data_orig))
l = round(0.03 * samplerate)
audio_chunks = list(get_audio_chunks(data, l))

test_block = audio_chunks[349]
result = lscgnm_driver(test_block, samplerate, l)


| sigma: 0.1 | gamma: -1 |
| sigma: 0.2 | gamma: -1 |
| sigma: 0.3 | gamma: -1 |
| sigma: 0.4 | gamma: 0.46337890625 |
| sigma: 0.5 | gamma: 0.91357421875 |
| sigma: 0.6 | gamma: 1.30908203125 |
| sigma: 0.7 | gamma: 1.69482421875 |
| sigma: 0.8 | gamma: 2.06884765625 |
| sigma: 0.9 | gamma: 2.44580078125 |
| sigma: 1.0 | gamma: 2.87744140625 |
